In [1]:
# importando bibliotecas
library('httr')
library("jsonlite")

In [2]:
# montando a carteira. Depois vamos pegar de algum lugar ou montar uma carteira mais real
carteiraJSON <- '[
  {"symbol" : "ALPA4", "Position" : 790000, "Price" : 12.22}
]' 

carteira <- fromJSON(carteiraJSON)

In [3]:
#pegando histórico de variação dos precos
query <- "http://api.informativos.io/prices_matrix?start_date=2018-08-13&end_date=2018-08-24&symbols[]=ALPA4"
response <- content(GET(url=query), "text")

# transformando num dataframe
data <- as.data.frame(fromJSON(response, simplifyDataFrame = TRUE), stringsAsFactors=FALSE)

#ajustando os dados
normalize_data <- function(df) {
  names(df) <- as.character(unlist(df[1,]))
  df[-1,]
}
precos <- normalize_data(data)

#criando estrutura que irá guardar os dados aplicados
precos_aplicados <- data.frame("Data" = c(), "Ativo" = c(), "Financeiro" = c(), stringsAsFactors = FALSE)

# iterando sobre a carteira
for (linha_carteira in 1:nrow(carteira)) {
    ativo <- carteira[linha_carteira, "symbol"]
    preco_original <- carteira[linha_carteira, "Price"]
    posicao_original <- carteira[linha_carteira, "Position"]
    financeiro_original <- preco_original * posicao_original
    #iterando sobre os precos
    for (linha_preco in 1:nrow(precos)) {
        # Aplicando na carteira
        change <- sapply(precos[linha_preco, ativo], as.numeric)
        perda_e_ganho <- financeiro_original * change        
        precos_aplicados <- rbind(precos_aplicados, data.frame("Data"=precos[linha_preco, "Date"], "Ativo"=ativo, "Financeiro"=perda_e_ganho))
    }    
}

# ajustando indices do dataframe
rownames(precos_aplicados) <- 1:nrow(precos_aplicados)

#agrupando por dia e somando os retornos
perda_ganho_agrupado <- aggregate(Financeiro ~ Data,precos_aplicados,sum);

print(perda_ganho_agrupado)

#ordenando os dados
ordenado <- perda_ganho_agrupado[order(perda_ganho_agrupado$Financeiro),]

print(ordenado)

#calculando o Var de 95% e 99%
var <- quantile(ordenado$Financeiro, probs=c(0.05, 0.01)) 

#imprimindo resultado
print(var)




         Data Financeiro
1  2018-08-13   31834.46
2  2018-08-14  420420.21
3  2018-08-15 -235643.94
4  2018-08-16  -23374.82
5  2018-08-17 -242126.05
6  2018-08-20  260567.34
7  2018-08-21 -259215.13
8  2018-08-22  258295.65
9  2018-08-23   15722.80
10 2018-08-24  -62788.94
         Data Financeiro
7  2018-08-21 -259215.13
5  2018-08-17 -242126.05
3  2018-08-15 -235643.94
10 2018-08-24  -62788.94
4  2018-08-16  -23374.82
9  2018-08-23   15722.80
1  2018-08-13   31834.46
8  2018-08-22  258295.65
6  2018-08-20  260567.34
2  2018-08-14  420420.21
       5%        1% 
-251525.0 -257677.1 
